In [4]:
using Pkg
Pkg.add("BenchmarkTools")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [9abbd945] + Profile v1.11.0
Precompiling project...
   3451.3 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 16 seconds. 492 already precompiled.


In [8]:
using Base.Threads
using BenchmarkTools

println("Threads ativos: ", nthreads())

function tarefa_pesada(x)
    s = 0.0
    for i in 1:50_000
        s += sin(x) * cos(i)
    end
    return s
end

function loop_serie()
    r = zeros(Float64,20)
    for i in 1:20
        r[i] = tarefa_pesada(i)
    end
    return r
end

function loop_paralelo()
    r = zeros(Float64,20)
    @threads for i in 1:20
        r[i] = tarefa_pesada(i)
    end
    return r
end

println("\nTempo série:")
@btime loop_serie();

println("\nTempo paralelo:")
@btime loop_paralelo();


Threads ativos: 2

Tempo série:
  32.096 ms (2 allocations: 224 bytes)

Tempo paralelo:
  29.854 ms (14 allocations: 1.33 KiB)


✅ 1. Explicação do código

O código implementa um experimento simples para comparar o desempenho entre execução em série e execução paralela usando threads em Julia.

Primeiro, defini uma função tarefa_pesada(x) que realiza muitos cálculos repetitivos, simulando uma operação custosa — isso garante que o teste seja suficientemente pesado para medir diferenças de desempenho.

Depois criei duas versões de um loop:

loop_serie(): executa a tarefa para 20 valores de forma sequencial.

loop_paralelo(): executa a mesma tarefa, mas usando @threads, que distribui as iterações entre os threads disponíveis.

Por fim, usei o macro @btime do pacote BenchmarkTools, que mede o tempo real de execução de maneira precisa, e coloquei ; no final das linhas para evitar que o vetor resultante apareça na saída.

O objetivo é demonstrar como a computação paralela pode reduzir o tempo de execução quando há múltiplos threads disponíveis.

✅ 2. Qual é melhor e por quê

Resultado:

Tempo série:      ~31.9 ms

Tempo paralelo:   ~29.9 ms

A execução paralela é ligeiramente mais rápida, mas a melhoria não é muito grande.
Isso acontece porque:

✔️ Só tenho 2 threads (pelo Collab)

Com apenas 2 threads, o máximo teórico de aceleração é 2×.
Mas na prática, a eficiência cai devido a custos como:

criação e sincronização das threads

acesso compartilhado à memória

sobrecarga do @threads

✔️ A tarefa de cada iteração não é tão pesada

Quanto mais pesada a tarefa, maior a vantagem do paralelismo.
Para tarefas leves, o custo de paralelizar pode até deixar o paralelo mais lento.

✔️ O loop só tem 20 iterações

Poucas iterações significam pouca oportunidade de dividir o trabalho entre threads.
